<a href="https://colab.research.google.com/github/Srinithya18/2048-Game/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
dt=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SPAM-210331-134237.csv",encoding="ISO-8859-1")

In [3]:
data=np.array(dt)
# print(data.shape,data)
# print(data[:5])
data[:, [0, 1]] = data[:, [1, 0]]
# print(data.shape,data[1])
# print(data[:5])
# z=np.zeros((116,1),dtype=np.float64)
# print(z)
# data= np.hstack((data, np.zeros((data.shape[0], 1), dtype=np.float64)))
# np.append(data,z,axis=1)
print(data[0])

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'
 'ham']


In [5]:
def preprocess_data(data):
    print("Preprocessing data...")
    
    punc = string.punctuation           # Punctuation list
    sw = stopwords.words('english')
    for record in data:
        # Remove common punctuation and symbols
      for item in punc:
        record[0] = record[0].replace(item, "")
         # Lowercase all letters and remove stopwords 
      splittedWords = record[0].split()
      newText = ""
      for word in splittedWords:
        if word not in sw:
          word = word.lower()
          newText = newText + " " + word
        
      record[0] = newText
        
    print("flag 2: preprocessed data")        
    return data
data=preprocess_data(data)
# print(data)

Preprocessing data...
flag 2: preprocessed data


In [6]:
# import random
# random.shuffle(data)
# print(data[0])
# train_data = data[0 : int(len(data)/2)]
# test_data = data[int(len(data)/2) + 1 : -1]
# # test_data = data[-101 : -1]
# print(train_data[0])
# print(test_data[0])

In [21]:
def split_data(data):
    print("Splitting data...")
    
    features = data[:, 1]   # array containing all email text bodies
    labels = data[:, 0]     # array containing corresponding labels
    # print(labels)
    training_labels, test_labels, training_data, test_data =\
        train_test_split(features, labels, test_size = 0.27, random_state = 42)
    # print(training_data[0],'|', test_data[0],'|', training_labels[0],'|', test_labels[0])
    # print(len(training_data),'|', len(test_data),'|',len(training_labels),'|', len(test_labels))
    print(training_data[0],test_data[0])
    print("flag 3: splitted data")
    return training_data, test_data, training_labels, test_labels
# training_data, test_data, training_labels, test_labels = split_data(data)
# split_data(data)

In [7]:
def getSimilarity(record1, record2):
  len1=len(record1[0].split())
  len2=len(record2[0].split())
  num_common=0
  d={}
  for word in record1[0].split():
    if word not in d:
      d[word]=1
  for word in record2[0].split():
    if word in d:
      num_common+=1
  similarity=num_common/(len1*len2)**0.5
  return similarity

In [22]:
def findKNN(train_data,record,k,train_label):
  for i in range(0,len(train_data)):
    sim=getSimilarity(train_data[i],record)
    train_data[i][-1]=sim
  res=[]
  for i in range(k):
    max_sim=0
    max_sim_index=0
    
    # testing_label=[]
    for i in range(0,len(train_data)):
      if train_data[i][-1]>max_sim:
        max_sim=train_data[i][-1]
        max_sim_index=i
    train_data[max_sim_index][-1]=0
    res.append(train_data[max_sim_index])
    train_label.append(training_labels[max_sim_index])
  return res

In [23]:
def judge(knn):
  num_ham=0
  num_spam=0
  for r in range(len(knn)):
    if(train_label[r]=="ham"):
      num_ham+=1
    else:
      num_spam+=1
  # print(num_ham)
  # print(num_spam)
  return "ham" if num_ham > num_spam else "spam"


In [25]:
def get_count(text):
    wordCounts = dict()
    for word in text.split():
        if word in wordCounts:
            wordCounts[word] += 1
        else:
            wordCounts[word] = 1
    
    return wordCounts

In [26]:
def euclidean_difference(test_WordCounts, training_WordCounts):
    total = 0
    for word in test_WordCounts:
      if word in test_WordCounts and word in training_WordCounts:
        total += (test_WordCounts[word] - training_WordCounts[word])**2
        del training_WordCounts[word]
      else:
        total += test_WordCounts[word]**2
    for word in training_WordCounts:
      total += training_WordCounts[word]**2
    return total**0.5

#To get spam count and ham count of k nearest neighbours.we selected k values
*italicized text*

In [27]:
def get_class(selected_Kvalues):
    spam_count = 0
    ham_count = 0
    for value in selected_Kvalues:
        if value[0] == "spam":
            spam_count += 1
        else:
            ham_count += 1
    if spam_count > ham_count:
        return "spam"
    else:
        return "ham"


In [28]:
def knn_classifier(training_data, training_labels, test_data, K, tsize):
    print("Running KNN Classifier...")
    
    result = []
    counter = 1
  # word counts for training email
    training_WordCounts = [] 
    for training_text in training_data:
            training_WordCounts.append(get_count(training_text))
    for test_text in test_data:
        similarity = [] # List of euclidean distances
        test_WordCounts = get_count(test_text)  # word counts for test email
        # Getting euclidean difference 
        for index in range(len(training_data)):
            euclidean_diff =\
                euclidean_difference(test_WordCounts, training_WordCounts[index])
            similarity.append([training_labels[index], euclidean_diff])
        # Sort list in ascending order based on euclidean difference
        similarity = sorted(similarity, key = lambda i:i[1])
        selected_Kvalues = [] 
        for i in range(K):
            selected_Kvalues.append(similarity[i])
        # Predicting the class of email
        result.append(get_class(selected_Kvalues))
    return result

Accuracy_score got using cosine distance.
**bold text**

In [31]:
correct=0
wrong=0
k=11
training_data, test_data, training_labels, test_labels = split_data(data)
x=[0.0]*len(training_data)
y=[0.0]*len(test_data)
train_data= np.column_stack((training_data, x))
testing_data= np.column_stack((test_data, y))
for d in range(len(testing_data)):
  train_label=[]
  knn=findKNN(train_data,testing_data[d],k,train_label)
  if judge(knn)==test_labels[d]:
    correct+=1
  else:
    wrong+=1
    # print(list(judge(knn)))
    # print(d[1])
accurate=correct/(correct+wrong)
print("correct",correct)
print("wrong",wrong)
print("accuracy ",accurate*100)

Splitting data...
 sorry ill call later  place man
flag 3: splitted data
correct 25
wrong 7
accuracy  78.125


Accuracy_score using euclidean_distance

In [30]:
def main(K):
    # data = load_data()
    # data = preprocess_data(data)
    training_data, test_data, training_labels, test_labels = split_data(data)
    tsize = len(test_data)
    result = knn_classifier(training_data, training_labels, test_data[:tsize], K, tsize)
    # for i in range(tsize):
      # print(test_labels[:tsize][i], result[i]) 
    accuracy = accuracy_score(test_labels[:tsize], result)
    # print(test_labels[:tsize],np.array(result))
    # precision = precision_score(test_labels[:tsize], result, pos_label="ham")
    # recall = recall_score(test_labels[:tsize], result, pos_label="ham")
    print("training data size\t: " + str(len(training_data)))
    print("test data size\t\t: " + str(len(test_data)))
    print("K value\t\t\t\t: " + str(K))
    print("Samples tested\t\t: " + str(tsize))
    print("% accuracy\t\t\t: " + str(accuracy * 100))
    # print("% precision\t\t\t: " + str(precision * 100))
    # print("% recall\t\t\t: " + str(recall * 100))
    print("Number correct\t\t: " + str(int(accuracy * tsize)))
    print("Number wrong\t\t: " + str(int((1 - accuracy) * tsize)))
main(30)

Splitting data...
 sorry ill call later  place man
flag 3: splitted data
Running KNN Classifier...
training data size	: 84
test data size		: 32
K value				: 30
Samples tested		: 32
% accuracy			: 71.875
Number correct		: 23
Number wrong		: 9
